In [ ]:
import shopify
import os
import pandas as pd
import json
from dotenv import load_dotenv

# Carregar variáveis do arquivo .env
load_dotenv()

API_KEY = os.getenv("SHOPIFY_API_KEY")
PASSWORD = os.getenv("SHOPIFY_PASSWORD")
SHOP_NAME = os.getenv("SHOPIFY_SHOP_NAME")
API_VERSION = os.getenv("SHOPIFY_API_VERSION")

# Configuração do SDK da Shopify
shop_url = f"https://{API_KEY}:{PASSWORD}@{SHOP_NAME}.myshopify.com/admin/api/{API_VERSION}"
shopify.ShopifyResource.set_site(shop_url)

# Função para testar a conexão
def test_connection() -> bool:
    try:
        shop = shopify.Shop.current()
        print(f"Conexão bem-sucedida! Nome da loja: {shop.name}")
        return True
    except Exception as e:
        print(f"Erro ao conectar à API Shopify: {e}")
        return False

# Obter todos os SKUs e IDs de produtos
def get_all_skus_with_product_ids() -> dict:
    try:
        sku_variants = {}
        page = 1
        while True:
            products = shopify.Product.find(limit=250, page=page)
            if not products:
                break
            for product in products:
                product_id = product.id
                vendor = product.vendor
                for variant in product.variants:
                    sku = variant.sku.strip()
                    if sku:
                        if sku not in sku_variants:
                            sku_variants[sku] = []
                        sku_variants[sku].append({
                            'variant_id': variant.id,
                            'product_id': product_id,
                            'vendor': vendor,
                            'inventory_item_id': variant.inventory_item_id
                        })
            page += 1
        return sku_variants
    except Exception as e:
        print(f"Erro ao obter SKUs: {e}")
        return {}

# Atualizar produto
def update_product(product_id: int, product_data: dict) -> bool:
    try:
        product = shopify.Product.find(product_id)
        for key, value in product_data.items():
            setattr(product, key, value)
        product.save()
        print(f"Produto {product_id} atualizado com sucesso.")
        return True
    except Exception as e:
        print(f"Erro ao atualizar o produto {product_id}: {e}")
        return False

# Atualizar variante
def update_variant(variant_id: int, variant_data: dict) -> bool:
    try:
        variant = shopify.Variant.find(variant_id)
        for key, value in variant_data.items():
            setattr(variant, key, value)
        variant.save()
        print(f"Variante {variant_id} atualizada com sucesso.")
        return True
    except Exception as e:
        print(f"Erro ao atualizar a variante {variant_id}: {e}")
        return False

# Atualizar imagens
def update_images(product_id: int, image_urls: list) -> bool:
    try:
        product = shopify.Product.find(product_id)
        # Limpar imagens existentes
        product.images = []
        # Adicionar novas imagens
        product.images = [{'src': url} for url in image_urls]
        product.save()
        print(f"Imagens do produto {product_id} atualizadas com sucesso.")
        return True
    except Exception as e:
        print(f"Erro ao atualizar imagens do produto {product_id}: {e}")
        return False

# Atualizar níveis de inventário
def update_inventory_level(inventory_item_id: int, quantity: int) -> bool:
    try:
        location = shopify.Location.find()[0]
        inventory_level = shopify.InventoryLevel.find_first(
            inventory_item_ids=inventory_item_id,
            location_ids=location.id
        )
        if inventory_level:
            inventory_level.available = quantity
            inventory_level.save()
            print(f"Nível de inventário atualizado para {inventory_item_id}.")
        else:
            print(f"Inventário para {inventory_item_id} não encontrado.")
        return True
    except Exception as e:
        print(f"Erro ao atualizar inventário para {inventory_item_id}: {e}")
        return False

# Criar novo produto
def create_product(product_data: dict) -> dict | None:
    try:
        product = shopify.Product()
        for key, value in product_data.items():
            setattr(product, key, value)
        product.save()
        print(f"Produto '{product_data.get('title')}' criado com sucesso.")
        return product
    except Exception as e:
        print(f"Erro ao criar o produto '{product_data.get('title')}': {e}")
        return None

# Função principal para processar e ingerir produtos
def process_and_ingest_products(df: pd.DataFrame) -> None:
    if not test_connection():
        print("Não foi possível conectar à Shopify. Encerrando...")
        return

    sku_data = get_all_skus_with_product_ids()
    for _, row in df.iterrows():
        product_data = {
            "title": row.get("title"),
            "body_html": row.get("description", ""),
            "vendor": row.get("vendor", ""),
            "tags": row.get("tags", ""),
        }
        variant_data = {
            "price": str(row.get("price", "0")),
            "sku": row.get("sku"),
            "inventory_quantity": row.get("inventory_quantity", 0),
        }
        sku = row.get("sku")
        if sku in sku_data:
            # Atualizar produto existente
            product_id = sku_data[sku][0]['product_id']
            variant_id = sku_data[sku][0]['variant_id']
            update_product(product_id, product_data)
            update_variant(variant_id, variant_data)
        else:
            # Criar novo produto
            product_data["variants"] = [variant_data]
            create_product(product_data)

# Exemplo de uso
if __name__ == "__main__":
    # Exemplo de DataFrame
    data = {
        "sku": ["123", "456"],
        "title": ["Produto A", "Produto B"],
        "description": ["Descrição A", "Descrição B"],
        "vendor": ["Marca A", "Marca B"],
        "tags": ["Tag1, Tag2", "Tag3, Tag4"],
        "price": [100.0, 200.0],
        "inventory_quantity": [10, 20]
    }
    df = pd.DataFrame(data)
    process_and_ingest_products(df)
